# What patterns exist between energy consumption and generation?

## Goals

By the end of this case, you will be even more familiar with using `pandas` for data transformation. You'll also gain experience with how to generate data visualizations like heat maps, line plots, strip plots, and box plots in Python. For these, you'll use the `matplotlib` and `seaborn` libraries.

## Inspecting the datasets

Let's start by importing our packages:

In [ ]:
# Load packages
import os
import pandas as pd
import numpy as np

# Required for basic python plotting functionality
import matplotlib.pyplot as plt

# Advanced plotting functionality with seaborn
import seaborn as sns

sns.set(style="whitegrid")  # can set style depending on how you'd like it to look

And now the CSV files:

In [ ]:
# Load the data into python
energy_df = pd.read_csv("data/energy_consumption.csv")
electricity_df = pd.read_csv("data/electricity_generation.csv")

As usual, we take a look at the head of each dataset:

In [ ]:
# Look at energy (consumption) data
energy_df.head()

In [ ]:
# Look at electricity (generation) data
electricity_df.head()

## Pre-processing data to simplify analysis moving forward

Data cleaning (i.e. data wrangling or data munging) is one of the most important first steps that you take when you analyze a dataset. Our energy dataset was not an exception to this. Recall what we had to do in lecture:

1. Remove missing values.
2. Shorten the descriptions.
3. Remove unnecessary columns.
4. Split the `YYYYMM` column.
5. Understand month 13 and remove it from the dataset.

Let's go over these one-by-one:

### Removing missing values

Before you remove missing values, you first need to detect them. The hint that made us realize we had missing data was that the `Value` column, instead of being of type `float64` as it would be logical, was of type `object`. That rose our suspicions, which were later confirmed when we found out that hidden amongst the numbers there was a string, `Not Available`, that made Python treat this numeric feature as a text feature.

To inspect the data types of a DataFrame, you can use the **`.dtypes`** attribute (it is not a method, so you don't add parenthesis):

In [ ]:
electricity_df.dtypes

A nifty hack to detect if a numeric column contains text is by sorting it. Here we use **`.sort_values()`**. Text values will then appear either at the top or the bottom of the sorted Series:

In [ ]:
electricity_df["Value"].sort_values() # To Z-A sort, put ascending=False inside the parenthesis

There's definitely some text there!

Let's replace the `Not Available` string with a native missing value representation. Here we'll use `numpy`'s `np.nan`. (An even better option is `pandas`'s `pd.NA`, but this one only works in newer versions of `pandas`.) To replace values in a `pandas` Series, you use, quite unsurprisingly, the **`.replace()`** method (the first argument is the value you are replacing, and the second argument is the value you are replacing it with):

In [ ]:
electricity_df["Value"] = electricity_df["Value"].replace("Not Available", np.nan)

### Finishing up with the `Value` column

Let's see if we succeeded. For this, we will use the **`.isna()`** and **`.value_counts()`** methods. `.isna()` creates a boolean Series in which there is a `True` if the corresponding value in `Value` is a `np.nan`, and a `False` otherwise. Then, `.value_counts()` counts the instances of `True` and `False` in that Boolean Series. This way, we can easily detect that there are 830 missing cells in our `Value` column:

In [ ]:
electricity_df["Value"].isna().value_counts()

Now, we use the **`.dropna()`** method to remove all rows in which at least one entry is missing:

In [ ]:
energy_df = energy_df.dropna()
electricity_df = electricity_df.dropna()

Finally, we convert `Value` to the `float64` data type, and we're done with our first task:

In [ ]:
electricity_df["Value"] = electricity_df["Value"].astype(float)

In [ ]:
electricity_df.dtypes

We've just introduced a lot of new methods! Let's take a moment to recap:

* `.dtypes` lets you access the data types of the columns of a DataFrame.
* `.sort_values()` sorts a Series.
* `.replace()` is used to replace values (including replacing strings with the `np.nan` missing value representation).
* `.isna()` and `.value_counts()` used together can tell us how many missing values we have in a Series.

### Renaming descriptions

Did you know that `.replace()` can also take dictionaries as input? This allows us to replace lots of `{original:replacement}` pairs with a single operation. 

### Example 1

Use the `.replace()` method with the below dictionaries to rename the values in the `Description` columns of `energy_df` and `electricity_df`.

In [ ]:
energy_short_dict = {
    "Primary Energy Consumed by the Commercial Sector": "PEC Commercial Sector",
    "Primary Energy Consumed by the Electric Power Sector": "PEC Electric Power Sector",
    "Primary Energy Consumed by the Industrial Sector": "PEC Industrial Sector",
    "Primary Energy Consumed by the Residential Sector": "PEC Residential Sector",
    "Primary Energy Consumed by the Transportation Sector": "PEC Transportation Sector",
    "Primary Energy Consumption Total": "PEC Total",
    "Total Energy Consumed by the Commercial Sector": "TEC Commercial Sector",
    "Total Energy Consumed by the Industrial Sector": "TEC Industrial Sector",
    "Total Energy Consumed by the Residential Sector": "TEC Residential Sector",
    "Total Energy Consumed by the Transportation Sector": "TEC Transportation Sector",
}


electricity_short_dict = {
    "Electricity Net Generation From Coal, All Sectors": "ENG Coal",
    "Electricity Net Generation From Conventional Hydroelectric Power, All Sectors": "ENG HE Power",
    "Electricity Net Generation From Geothermal, All Sectors": "ENG Geothermal",
    "Electricity Net Generation From Hydroelectric Pumped Storage, All Sectors": "ENG HE Pumped Storage",
    "Electricity Net Generation From Natural Gas, All Sectors": "ENG Natural Gas",
    "Electricity Net Generation From Nuclear Electric Power, All Sectors": "ENG Nuclear Electric Power",
    "Electricity Net Generation From Other Gases, All Sectors": "ENG Other Gases",
    "Electricity Net Generation From Petroleum, All Sectors": "ENG Petroleum",
    "Electricity Net Generation From Solar, All Sectors": "ENG Solar",
    "Electricity Net Generation From Waste, All Sectors": "ENG Waste",
    "Electricity Net Generation From Wind, All Sectors": "ENG Wind",
    "Electricity Net Generation From Wood, All Sectors": "ENG Wood",
    "Electricity Net Generation Total (including from sources not shown), All Sectors": "ENG Total",
}

**Answer.** Shown below:

In [ ]:
energy_df["Description"] = energy_df["Description"].replace(energy_short_dict)
electricity_df["Description"] = electricity_df["Description"].replace(electricity_short_dict)

### Exercise 1

Create a replacement dictionary and then change `Trillion Btu` in `energy_df` for `Trillion British Thermal Units`.

**Answer.**

-------

### Removing unnecessary columns

This one is even easier than the previous two. To drop a column from a DataFrame, you use the **`.drop()`** method:

In [ ]:
energy_df = energy_df.drop(columns=["MSN", "Column_Order"])
electricity_df = electricity_df.drop(columns=["MSN", "Column_Order"])

# Making sure it worked -- hooray!
energy_df.head()

### Splitting the `YYYYMM` column

Sometimes you have IDs or dates in a format that contains valuable but disparate bits of information squeezed together as a single string. For instance:

* `FordMustang2019`. This contains three features: `Ford` (make), `Mustang` (model), and `2019` (year).
* `SamsungGalaxy`. This has two: `Samsung` (make) and `Galaxy` (model).
* `199901`. Here we have a year (`1999`) and a month (`01`). This is what we have in this dataset.

One clever way to separate the features is by using string methods in `pandas`. You access them appending `.str` to the Series and then calling the method as such. In this case, we want to split `199901` into `1999` and `01`, so one good strategy would be to split the string right after the fourth character. You might remember from a previous case that if you want to take the first 4 characters of a string, you can slice it with `[0:4]`. Let's do that and get the years (we had first to convert `YYYYMM` into a string column because it was originally stored as an integer column):

In [ ]:
energy_df["YYYYMM"] = energy_df["YYYYMM"].astype(str)
energy_df["YYYY"] = energy_df["YYYYMM"].str[0:4] # Notice that we wrote .str before slicing
energy_df.head()

Let's do the same for the `electricity_df` DataFrame:

In [ ]:
electricity_df["YYYYMM"] = electricity_df["YYYYMM"].astype(str)
electricity_df["YYYY"] = electricity_df["YYYYMM"].str[0:4]

### Example 2

Get the months and add them to both DataFrames as the `MM` column.

**Hint:** You can use [negative slicing with strings](https://www.w3schools.com/python/gloss_python_string_negative_indexing.asp) as well!

**Answer.** Shown below:

In [ ]:
energy_df["MM"] = energy_df["YYYYMM"].str[-2:]
electricity_df["MM"] = electricity_df["YYYYMM"].str[-2:]

energy_df.head()

Let's now remove the month 13 rows, since those are simply the sum of all the other values for each year. We can take advantage of the `.drop()` method again, because not only it can drop columns, it can also drop rows:

In [ ]:
dropme_energy = energy_df["MM"]=="13" # We filter the rows with month 13
energy_df = energy_df.drop(energy_df[dropme_energy].index)
energy_df.head()

Here, we first defined the condition `dropme_energy`, which is a Boolean Series. Then we filtered our DataFrame according to that condition and extracted the indices of the rows that we wanted to remove. We did that with `energy_df[dropme_energy].index`. And finally, we passed those indices to the `.drop()` function.

We now do the same with the `electricity_df` DataFrame:

In [ ]:
dropme_electricity = electricity_df["MM"]=="13" # We filter the rows with month 13
electricity_df = electricity_df.drop(electricity_df[dropme_electricity].index)
electricity_df.head()

## Identifying the relationship between energy consumption and generation

Here's how to make the `PEC Electric Power Sector` vs. `ENG Nuclear Electric Power` scatterplot using `matplotlib`. First, we filter the DataFrames to keep only the rows we need:

In [ ]:
# Filtering the DataFrames
consume_df = energy_df[energy_df["Description"] == "PEC Electric Power Sector"]
generate_df = electricity_df[electricity_df["Description"] == "ENG Nuclear Electric Power"]

# Getting rid of unnecesary columns
consume_df = consume_df.drop(columns=["Description", "Unit", "YYYY", "MM"])
generate_df = generate_df.drop(columns=["Description", "Unit", "YYYY", "MM"])

Then, we merge the DataFrames using the **`pd.merge()`** function. To join two DataFrames this way, there has to be a column with common values in both (this column is called the **join key**). Our datasets have all their columns in common, but the one we will use is `YYYYMM` because in order to make our scatterplot we need both datasets to be aligned by their dates:

In [ ]:
merged_df = pd.merge(consume_df, generate_df, on="YYYYMM")
merged_df.head()

As you see, we got two `Value` columns , but we don't know which one is consumption and which one is generation. To fix this, we use the `suffixes` argument:

In [ ]:
merged_df = pd.merge(consume_df, generate_df, on="YYYYMM", suffixes=["_CONSUME", "_GENERATE"])
merged_df.head()

This is the code to make the plot (`plt` is the alias we used to import `matplotlib`'s plotting module):

In [ ]:
plt.scatter(x = merged_df["Value_GENERATE"], y = merged_df["Value_CONSUME"]) # The plot
plt.title("Nuclear Electric Power Analysis") # Adding a title
plt.xlabel("ENG Nuclear Electric Power") # Adding axis labels
plt.ylabel("PEC Electric Power Sector")

#### A word about `pd.merge()`

The `pd.merge()` function will definitely be one the most important elements in your toolbox in the future, so it makes sense that we explain it a little more here. When you merge two DataFrames you are joining them by their shared key. There are four main kinds of merge operations in `pandas`: left joins, right joins, inner joins, and full joins. Let's explain them graphically.

This will be our `left` DataFrame (we called it `L`):

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>number_plate</th>      <th>has_tickets</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>IBV2750</td>      <td>True</td>    </tr>    <tr>      <th>1</th>      <td>EUS687</td>      <td>False</td>    </tr>    <tr>      <th>2</th>      <td>AYE7756</td>      <td>True</td>    </tr>  </tbody></table>

And this will be our `right` DataFrame (`R`):

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>number_plate</th>      <th>owners_name</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>6TRJ244</td>      <td>Michael Holmes</td>    </tr>    <tr>      <th>1</th>      <td>EUS687</td>      <td>Jenny Liu</td>    </tr>    <tr>      <th>2</th>      <td>532484</td>      <td>Geoffrey Frank</td>    </tr>  </tbody></table>

##### Left join

Arguably the most common way of merging DataFrames is using a **left join**, that graphically looks like this:
~~~python
pd.merge(L, R, on="number_plate", how="left")
~~~

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>number_plate</th>      <th>has_tickets</th>      <th>owners_name</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>IBV2750</td>      <td>True</td>      <td>NaN</td>    </tr>    <tr>      <th>1</th>      <td>EUS687</td>      <td>False</td>      <td>Jenny Liu</td>    </tr>    <tr>      <th>2</th>      <td>AYE7756</td>      <td>True</td>      <td>NaN</td>    </tr>  </tbody></table>

Here we took all the elements of the `left` DataFrame and tried to find a match in the `right` DataFrame for each one of them. Then we reported all the elements of the `left` DataFrame regardless of whether we did find a match or not. In this case, the only car that had a match was `EUS687` and hence it is the only one that has an associated `owners_name` in the result.

##### Right join

This is exactly the same as the left join, with the difference that we report all the elements of the `right` DataFrame instead of those of the `left` DataFrame:

~~~python
pd.merge(L, R, on="number_plate", how="right")
~~~

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>number_plate</th>      <th>has_tickets</th>      <th>owners_name</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>6TRJ244</td>      <td>NaN</td>      <td>Michael Holmes</td>    </tr>    <tr>      <th>1</th>      <td>EUS687</td>      <td>False</td>      <td>Jenny Liu</td>    </tr>    <tr>      <th>2</th>      <td>532484</td>      <td>NaN</td>      <td>Geoffrey Frank</td>    </tr>  </tbody></table>

This would be equivalent to `pd.merge(R, L, on="number_plate", how="left")` (notice that we swapped the DataFrames).

##### Inner join

This kind of join only reports those elements that are both in the `right` and the `left` DataFrames:

~~~python
pd.merge(L, R, on="number_plate", how="inner")
~~~

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>number_plate</th>      <th>has_tickets</th>      <th>owners_name</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>EUS687</td>      <td>False</td>      <td>Jenny Liu</td>    </tr>  </tbody></table>

##### Full join

Finally, a **full join** reports all the elements of both DataFrames, including those that had a match and those that did not:

~~~python
pd.merge(L, R, on="number_plate", how="outer")
~~~

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>number_plate</th>      <th>has_tickets</th>      <th>owners_name</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>IBV2750</td>      <td>True</td>      <td>NaN</td>    </tr>    <tr>      <th>1</th>      <td>EUS687</td>      <td>False</td>      <td>Jenny Liu</td>    </tr>    <tr>      <th>2</th>      <td>AYE7756</td>      <td>True</td>      <td>NaN</td>    </tr>    <tr>      <th>3</th>      <td>6TRJ244</td>      <td>NaN</td>      <td>Michael Holmes</td>    </tr>    <tr>      <th>4</th>      <td>532484</td>      <td>NaN</td>      <td>Geoffrey Frank</td>    </tr>  </tbody></table>

Keep in mind that the keyword for a full join in `pandas` is not ~`full`~ but `outer`. This is because "full join" is a shortened form of "full outer join".

### Exercise 2

Copy the code of the previous cells and paste it below. Then modify it to make a scatterplot of `PEC Commercial Sector` vs. `ENG Nuclear Electric Power` this time.

**Answer.**

-------

## Trends in energy consumption and generation over time

While a scatterplot helps us visualize the relationship between two variables, it does not allow us to look at something across time. For this, we will use the line plot. A line plot is excellent for viewing time series data and will help us determine trends and cyclical patterns across time for both electric power sector energy consumption and nuclear electric power energy generation.

Let's build a line plot for the `PEC Electric Power Sector` description:

In [ ]:
# Filtering the DataFrame
consume_df = energy_df[energy_df["Description"] == "PEC Electric Power Sector"].copy()
plt.plot(consume_df["Value"]) # plt.plot is the matplotlib function to draw line plots

This plot looks okay, but the horizontal axis is not very meaningful. Its labels are numbers that don't seem to be dates. A quick inspection of our data reveals that these are simply the row indices of the DataFrame:

In [ ]:
consume_df.head()

In order to put the dates on the horizontal axis, we need to convert the `YYYYMM` column to `pandas`'s native `datetime` data type. We've done this before in another case, but here's again how to do it:

In [ ]:
converted_date = pd.to_datetime(consume_df["YYYYMM"], format="%Y%m")
consume_df["Date"] = converted_date
consume_df.head()

Let's explain what we've just done. First, we used `pd.to_datetime()` to convert `consume_df["YYYYMM"]` into the `datetime` type. The `format="%Y%m"` argument is simply telling `pandas` that the `YYYYMM` column starts with the year (the `%Y` part) and then has the month (the `%m` part), and does not include information about the day.

Now equipped with our newly created `Date` column, we can remake the plot and add a title and axis labels:

In [ ]:
plt.figure(figsize=(15, 3)) # To make it larger
plt.plot(consume_df["Date"], consume_df["Value"]) # The order is plt.plot(x,y)
plt.title("Line Plot: PEC Electric Power Sector")
plt.xlabel("Date")
plt.ylabel("Value - PEC Electric Power Sector")

We can even make a plot of percentage variations. For that, we will use the handy [**`.pct_change()`**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html) Series method:

In [ ]:
plt.figure(figsize=(15, 3)) # To make it larger
plt.plot(consume_df["Date"], consume_df["Value"].pct_change()) # The order is plt.plot(x,y)
plt.title("Line Plot: PEC Electric Power Sector")
plt.xlabel("Date")
plt.ylabel("Value - PEC Electric Power Sector")

### Exercise 3

Repeat the steps above, but make a line plot of `ENG Nuclear Electric Power` instead. Make the percentage change plot too.

**Hint:** Here's the first line of the answer to help you get started:

~~~python
generate_df = electricity_df[electricity_df["Description"] == "ENG Nuclear Electric Power"]
~~~

**Answer.**

-------

## Using ```seaborn```


While ```matplotlib``` is a nice plotting package, it is meant to be a general plotting tool. ```seaborn``` is an extremely useful data visualization library specifically designed for statistical figures. We will go through some important plotting functionality while continuing to identify trends in the energy data. We will only be scratching the surface of ```seaborn```, but [here](https://seaborn.pydata.org/api.html) is a list of all the possible plots.

### Heat maps

Let's now look at consumption and generation levels month-by-month over time to see if the peak cyclical patterns we see are stable across many decades of data.

```seaborn``` offers the powerful heat map method [**```heatmap()```**](https://seaborn.pydata.org/generated/seaborn.heatmap.html#seaborn.heatmap) that will allow us to nicely visualize the monthly energy consumed and electricity generated over time. Before we use it, let's first create a pivot table as usual:

In [ ]:
generate_df = electricity_df[electricity_df["Description"] == "ENG Nuclear Electric Power"]
pivoted_df = pd.pivot_table(data=generate_df, index="MM", columns="YYYY", values="Value")
pivoted_df

Now it's simply a matter of using `sns.heatmap()`. You can customize the plot (like setting its size and adding a title and axis labels) using `matplotlib` just as with the previous plots, because `seaborn` is built on top of `matplotlib`:

In [ ]:
plt.figure(figsize=(15, 3)) # We can set the size using matplotlib
sns.heatmap(pivoted_df, cmap="coolwarm") # cmap defines the color palette

### Box plots

We can create box plots just as easily with the `sns.boxplot()` function. In the cell below, we ask `seaborn` to group our data by `Description` (the x-axis) and plot the distribution of `Value` for each `Description`. The `plt.xticks(rotation=90)` line is a bit of `matplotlib` code that rotates the x-axis labels by 90 degrees to prevent label collisions:

In [ ]:
sns.boxplot(x="Description", y="Value", data=energy_df)
plt.xticks(rotation=90)

The size of some `seaborn` plots won't be affected by `plt.figure(figsize=())`. This is the case for box plots - the syntax is just slightly different:

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
sns.boxplot(x="Description", y="Value", data=energy_df, ax=ax)
plt.xticks(rotation=90)

### Strip plots

### Exercise 4 (optional)

Making strip plots is very easy - you use `sns.stripplot()`. Copy the previous code cell and modify it to make a strip plot instead of a box plot.

**Answer.**

-------

## Appendix

Here is a summary of the functions and tools introduced in this workshop:

* `pandas` data exploration functions 
    * [`dtypes`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html): This is an attribute of `pandas` DataFrames that tells you the data types of the columns of the DataFrame. We don't use parenthesis when we access it.
    * [`.isna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isna.html): When you call this function on a DataFrame or Series, it outputs a boolean object whose values are `True` if the corresponding value in the DataFrame is a null value, and `False` otherwise.
    * [`.value_counts()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html): This Series method creates a frequency table of the values in a Series.
* `pandas` data transformation functions
    * [`.drop()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html): Drops either columns or rows in a DataFrame. To drop columns, you pass the `columns` argument and a list of the columns you want to remove. To drop rows, you do the same, only using the argument `index` instead.
    * [`.dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html): This method finds all the rows in a DataFrame or Series that contain null fields and removes them. If you call it on a DataFrame, you can optionally specify which columns to look for null values in, so that null values in other columns are allowed (for this, you use the `subset` argument).
    * `.pct_change()`: This method computes the percentage change in a Series, comparing each item with the previous one (other lags are [possible](https://pandas.pydata.org/docs/reference/api/pandas.Series.pct_change.html)).
    * [`.replace()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html): Call this method to replace values in a DataFrame or Series. You can pass a dictionary.
    * [`.sort_values()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html): To sort a DataFrame or Series. You use `by` to tell `pandas` which column you would like to sort by, and to do a Z-A sort (or, equivalently, a 9-0 sort) you pass `ascending=False`.
* `seaborn` plotting functions
    * [`.boxplot()`](https://seaborn.pydata.org/generated/seaborn.boxplot.html) for box plots
    * [`.heatmap()`](https://seaborn.pydata.org/generated/seaborn.heatmap.html) for heat maps
    * [`.stripplot()`](https://seaborn.pydata.org/generated/seaborn.stripplot.html) for strip plots